# Data Cleaning / Preprocessing / Analysis

In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
# Import libraries/frameworks
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [3]:
df = pd.read_csv('/content/BEA 2024 Task Data Extended_new.csv')
df.head()

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time
0,622,A 27-year-old man comes to the emergency depar...,Choristoma,Ciliary epithelium,Endothelial cells,Gastric mucosa,Striated muscle,NaN,NaN,NaN,NaN,NaN,D,Gastric mucosa,Text,STEP 1,0.38,123.96
1,440,"A 39-year-old woman, gravida 2, para 2, comes ...",Future fertility plans,Hypertension,Obesity,Patient age,Stage of disease,NaN,NaN,NaN,NaN,NaN,E,Stage of disease,Text,STEP 3,0.50,77.53
2,12,An 18-year-old primigravid woman comes for her...,Repeat measurement of MSAFP concentration,"Triple screening for MSAFP, serum ÃŽÂ²-hCG, an...",Ultrasonography,Amniocentesis for measurement of ÃŽÂ±-fetoprot...,Amniocentesis for chromosomal analysis,NaN,NaN,NaN,NaN,NaN,C,Ultrasonography,Text,STEP 2,0.58,65.15
3,5,"A 26-year-old man, who is admitted to the hosp...",There is no evidence of a familial coagulation...,Half of their daughters will have a clinically...,The sons of their daughters will be at risk fo...,Their sons will be at risk for a clinically ev...,Their sons and daughters will be at risk for a...,NaN,NaN,NaN,NaN,NaN,C,The sons of their daughters will be at risk fo...,Text,STEP 3,0.56,105.24
4,221,A 60-year-old man had a total thyroidectomy an...,Babinski sign present bilaterally,Chvostek sign,Deviation of the tongue to the left side,A drooping left shoulder,Hyporeflexia,NaN,NaN,NaN,NaN,NaN,B,Chvostek sign,Text,STEP 3,0.22,57.02


In [4]:
df.fillna(0, inplace=True)
df.head(2)

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time
0,622,A 27-year-old man comes to the emergency depar...,Choristoma,Ciliary epithelium,Endothelial cells,Gastric mucosa,Striated muscle,0,0,0,0,0,D,Gastric mucosa,Text,STEP 1,0.38,123.96
1,440,"A 39-year-old woman, gravida 2, para 2, comes ...",Future fertility plans,Hypertension,Obesity,Patient age,Stage of disease,0,0,0,0,0,E,Stage of disease,Text,STEP 3,0.50,77.53


In [5]:
cols = df.columns
cols

Index(['ItemNum', 'ItemStem_Text', 'Answer__A', 'Answer__B', 'Answer__C',
       'Answer__D', 'Answer__E', 'Answer__F', 'Answer__G', 'Answer__H',
       'Answer__I', 'Answer__J', 'Answer_Key', 'Answer_Text', 'ItemType',
       'EXAM', 'Difficulty', 'Response_Time'],
      dtype='object')

In [6]:
df.ItemType.value_counts()

ItemType
Text    595
PIX      72
Name: count, dtype: int64

In [7]:
df.EXAM.value_counts()

EXAM
STEP 1    273
STEP 2    220
STEP 3    174
Name: count, dtype: int64

In [8]:
X_cols = ['ItemNum', 'ItemStem_Text', 'Answer__A', 'Answer__B', 'Answer__C',
          'Answer__D', 'Answer__E', 'Answer__F', 'Answer__G', 'Answer__H',
          'Answer__I', 'Answer__J', 'Answer_Key', 'Answer_Text']
y_1 = df['Difficulty']
y_2 = df['Response_Time']

In [9]:
df[df['ItemNum'] == 285]

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time
451,285,A 45-year-old man comes to the office for an i...,Administer amobarbital and then interview the ...,Ask the patient to provide a narrative with de...,Interview the patient under hypnosis,Interview the patient while paying close atten...,Tell the patient he is exhibiting behaviors th...,0,0,0,0,0,B,Ask the patient to provide a narrative with de...,Text,STEP 3,0.16,56.0


In [10]:
df[X_cols] = df[X_cols].astype('str')

# Dividing data into train and test set

In [11]:
from sklearn.model_selection import train_test_split

# First, perform the split once to get the indices
X_train, X_test, y_train, y_test = train_test_split(df[X_cols], df.index, test_size=0.2, random_state=1)

# Use the indices to create the consistent splits for y_1 and y_2
y_train1, y_test1 = y_1[y_train], y_1[y_test]
y_train2, y_test2 = y_2[y_train], y_2[y_test]

# Optionally, you can name X_train and X_test to X_train1/X_train2 and X_test1/X_test2 for consistency
X_train1, X_test1 = X_train, X_test
X_train2, X_test2 = X_train, X_test

# Now, X_train1 and X_train2 are identical, as well as X_test1 and X_test2.


# Vectorizing input

In [12]:
# reference: https://huggingface.co/pritamdeka/S-PubMedBert-MS-MARCO
model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.55k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/388 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/461k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
# Example
from transformers import AutoTokenizer, AutoModel
import torch

# Mean Pooling - Take attention mask into account for correct averaging


def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(
        -1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'This is for NLP review']
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('pritamdeka/S-PubMedBert-MS-MARCO')
model = AutoModel.from_pretrained('pritamdeka/S-PubMedBert-MS-MARCO')
# Tokenize sentences
encoded_input = tokenizer(
    sentences, padding=True, truncation=True, return_tensors='pt')
# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
print("Sentence embeddings:")
print(sentence_embeddings)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence embeddings:
tensor([[-0.5003, -0.5050, -0.4711,  ...,  0.1225,  0.0509,  0.4718],
        [-0.4316, -0.7740, -0.3478,  ...,  0.2539,  0.7958,  0.4290]])


In [14]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np

model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')

def vectorizer(data):
    data.columns = data.columns.str.strip()
    data_vec = []
    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        ItemStem_Text_vec = model.encode(row['ItemStem_Text'])
        cnt = 0
        temp_vec = np.zeros(768)
        for option in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']:
            if row['Answer__' + option] != '0':
                cnt += 1
                try:
                    temp_vec += model.encode(row['Answer__' + option])
                except:
                    print(f"Error in {row['Answer__' + option]}")
                    cnt -= 1
        temp_vec /= cnt
        Answer_vec = model.encode(row['Answer_Text'])
        final_vec = ItemStem_Text_vec + temp_vec + Answer_vec
        data_vec.append(final_vec)
    return np.array(data_vec)

X_vec = vectorizer(df[X_cols])
X_vec.shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 667/667 [11:52<00:00,  1.07s/it]


(667, 768)

# Neural network - baseline model - predicting difficulty

In [15]:
scaler = StandardScaler()
X_vec_scaled = scaler.fit_transform(X_vec)

In [16]:
def build_dl_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(768,)),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),  # Adding dropout to prevent overfitting
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),  # Adding another dropout layer
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model


# Implement K-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
y1_predict_full = np.zeros(len(X_vec_scaled))
y2_predict_full = np.zeros(len(X_vec_scaled))
rmse_scores_1 = []

for train_index, test_index in kfold.split(X_vec_scaled):
    X_train, X_test = X_vec_scaled[train_index], X_vec_scaled[test_index]
    y_train1, y_test1 = y_1[train_index], y_1[test_index]
    y_train2, y_test2 = y_2[train_index], y_2[test_index]

    dl_model_1 = build_dl_model()
    dl_model_1.fit(X_train, y_train1, epochs=100)


    y1_predict = dl_model_1.predict(X_test)
    y1_predict_full[test_index] = y1_predict.flatten()
    mse = mean_squared_error(y_test1, y1_predict)
    rmse = np.sqrt(mse)
    rmse_scores_1.append(rmse)

print("RMSE for Difficulty with K-Fold Cross-Validation:", np.mean(rmse_scores_1))

Epoch 1/100
17/17 [==============================] - 1s 5ms/step - loss: 0.3961 - root_mean_squared_error: 0.6294
Epoch 2/100
17/17 [==============================] - 0s 4ms/step - loss: 0.1654 - root_mean_squared_error: 0.4066
Epoch 3/100
17/17 [==============================] - 0s 5ms/step - loss: 0.1217 - root_mean_squared_error: 0.3488
Epoch 4/100
17/17 [==============================] - 0s 4ms/step - loss: 0.1121 - root_mean_squared_error: 0.3349
Epoch 5/100
17/17 [==============================] - 0s 4ms/step - loss: 0.1067 - root_mean_squared_error: 0.3267
Epoch 6/100
17/17 [==============================] - 0s 6ms/step - loss: 0.0847 - root_mean_squared_error: 0.2910
Epoch 7/100
17/17 [==============================] - 0s 4ms/step - loss: 0.0794 - root_mean_squared_error: 0.2817
Epoch 8/100
17/17 [==============================] - 0s 4ms/step - loss: 0.0764 - root_mean_squared_error: 0.2764
Epoch 9/100
17/17 [==============================] - 0s 4ms/step - loss: 0.0667 - root_m

In [17]:
dl_model_1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 256)               196864    
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dense_22 (Dense)            (None, 64)                8256      
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_23 (Dense)            (None, 32)                2080      
                                                                 
 dense_24 (Dense)            (None, 1)                

In [18]:
y1_predict = dl_model_1.predict(X_vec)  # Predict on all data

for i, prediction in enumerate(y1_predict):
  print(f"Entry {i+1}: Predicted Difficulty = {prediction[0]:.4f}") # Display with 4 decimal places

21/21 [==============================] - 0s 2ms/step
Entry 1: Predicted Difficulty = 0.4783
Entry 2: Predicted Difficulty = 0.5065
Entry 3: Predicted Difficulty = 0.4658
Entry 4: Predicted Difficulty = 0.4535
Entry 5: Predicted Difficulty = 0.4181
Entry 6: Predicted Difficulty = 0.4572
Entry 7: Predicted Difficulty = 0.3914
Entry 8: Predicted Difficulty = 0.4266
Entry 9: Predicted Difficulty = 0.3903
Entry 10: Predicted Difficulty = 0.4603
Entry 11: Predicted Difficulty = 0.4476
Entry 12: Predicted Difficulty = 0.4125
Entry 13: Predicted Difficulty = 0.5034
Entry 14: Predicted Difficulty = 0.5690
Entry 15: Predicted Difficulty = 0.5107
Entry 16: Predicted Difficulty = 0.5141
Entry 17: Predicted Difficulty = 0.5027
Entry 18: Predicted Difficulty = 0.5228
Entry 19: Predicted Difficulty = 0.4577
Entry 20: Predicted Difficulty = 0.4270
Entry 21: Predicted Difficulty = 0.4832
Entry 22: Predicted Difficulty = 0.5803
Entry 23: Predicted Difficulty = 0.3626
Entry 24: Predicted Difficulty = 0.4

# Neural network - baseline model - predicting response time

In [19]:
rmse_scores_2 = []

for train_index, test_index in kfold.split(X_vec_scaled):
    X_train, X_test = X_vec_scaled[train_index], X_vec_scaled[test_index]
    y_train2, y_test2 = y_2[train_index], y_2[test_index]

    dl_model_2 = build_dl_model()
    dl_model_2.fit(X_train, y_train2, epochs=100, verbose=1)

    y2_predict = dl_model_2.predict(X_test)
    y2_predict_full[test_index] = y2_predict.flatten()
    mse = mean_squared_error(y_test2, y2_predict)
    rmse = np.sqrt(mse)
    rmse_scores_2.append(rmse)

print("RMSE for Response Time with K-Fold Cross-Validation:", np.mean(rmse_scores_2))

Epoch 1/100
17/17 [==============================] - 1s 5ms/step - loss: 6997.5220 - root_mean_squared_error: 83.6512
Epoch 2/100
17/17 [==============================] - 0s 4ms/step - loss: 1996.5251 - root_mean_squared_error: 44.6825
Epoch 3/100
17/17 [==============================] - 0s 4ms/step - loss: 798.7978 - root_mean_squared_error: 28.2630
Epoch 4/100
17/17 [==============================] - 0s 4ms/step - loss: 620.0202 - root_mean_squared_error: 24.9002
Epoch 5/100
17/17 [==============================] - 0s 4ms/step - loss: 452.1705 - root_mean_squared_error: 21.2643
Epoch 6/100
17/17 [==============================] - 0s 4ms/step - loss: 390.1005 - root_mean_squared_error: 19.7510
Epoch 7/100
17/17 [==============================] - 0s 4ms/step - loss: 320.5772 - root_mean_squared_error: 17.9047
Epoch 8/100
17/17 [==============================] - 0s 4ms/step - loss: 327.4988 - root_mean_squared_error: 18.0969
Epoch 9/100
17/17 [==============================] - 0s 4ms/st

In [20]:
dl_model_2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 256)               196864    
                                                                 
 dropout_18 (Dropout)        (None, 256)               0         
                                                                 
 dense_46 (Dense)            (None, 128)               32896     
                                                                 
 dense_47 (Dense)            (None, 64)                8256      
                                                                 
 dropout_19 (Dropout)        (None, 64)                0         
                                                                 
 dense_48 (Dense)            (None, 32)                2080      
                                                                 
 dense_49 (Dense)            (None, 1)                

In [21]:
y2_predict = dl_model_2.predict(X_vec)

for i, prediction in enumerate(y2_predict):
  print(f"Entry {i+1}: Predicted Difficulty = {prediction[0]:.4f}")

21/21 [==============================] - 0s 2ms/step
Entry 1: Predicted Difficulty = 145.5295
Entry 2: Predicted Difficulty = 145.3382
Entry 3: Predicted Difficulty = 125.4282
Entry 4: Predicted Difficulty = 140.4260
Entry 5: Predicted Difficulty = 127.9034
Entry 6: Predicted Difficulty = 153.3349
Entry 7: Predicted Difficulty = 131.9115
Entry 8: Predicted Difficulty = 127.3632
Entry 9: Predicted Difficulty = 125.8338
Entry 10: Predicted Difficulty = 134.3209
Entry 11: Predicted Difficulty = 130.3345
Entry 12: Predicted Difficulty = 133.9608
Entry 13: Predicted Difficulty = 154.3753
Entry 14: Predicted Difficulty = 154.0755
Entry 15: Predicted Difficulty = 126.2022
Entry 16: Predicted Difficulty = 146.9126
Entry 17: Predicted Difficulty = 137.4817
Entry 18: Predicted Difficulty = 142.1140
Entry 19: Predicted Difficulty = 112.1195
Entry 20: Predicted Difficulty = 121.5156
Entry 21: Predicted Difficulty = 120.0926
Entry 22: Predicted Difficulty = 149.3743
Entry 23: Predicted Difficulty =

# Add predicted values to dataframe

In [22]:
df['Predicted_Difficulty'] = y1_predict_full
df['Predicted_Response_Time'] = y2_predict_full

In [23]:
df

,ItemNum,ItemStem_Text,Answer__A,Answer__B,Answer__C,Answer__D,Answer__E,Answer__F,Answer__G,Answer__H,Answer__I,Answer__J,Answer_Key,Answer_Text,ItemType,EXAM,Difficulty,Response_Time,Predicted_Difficulty,Predicted_Response_Time
0,622,A 27-year-old man comes to the emergency depar...,Choristoma,Ciliary epithelium,Endothelial cells,Gastric mucosa,Striated muscle,0,0,0,0,0,D,Gastric mucosa,Text,STEP 1,0.38,123.96,0.471851,63.185619
1,440,"A 39-year-old woman, gravida 2, para 2, comes ...",Future fertility plans,Hypertension,Obesity,Patient age,Stage of disease,0,0,0,0,0,E,Stage of disease,Text,STEP 3,0.50,77.53,0.453215,114.198326
2,12,An 18-year-old primigravid woman comes for her...,Repeat measurement of MSAFP concentration,"Triple screening for MSAFP, serum ÃŽÂ²-hCG, an...",Ultrasonography,Amniocentesis for measurement of ÃŽÂ±-fetoprot...,Amniocentesis for chromosomal analysis,0,0,0,0,0,C,Ultrasonography,Text,STEP 2,0.58,65.15,0.363295,61.296341
3,5,"A 26-year-old man, who is admitted to the hosp...",There is no evidence of a familial coagulation...,Half of their daughters will have a clinically...,The sons of their daughters will be at risk fo...,Their sons will be at risk for a clinically ev...,Their sons and daughters will be at risk for a...,0,0,0,0,0,C,The sons of their daughters will be at risk fo...,Text,STEP 3,0.56,105.24,0.406027,82.920189
4,221,A 60-year-old man had a total thyroidectomy an...,Babinski sign present bilaterally,Chvostek sign,Deviation of the tongue to the left side,A drooping left shoulder,Hyporeflexia,0,0,0,0,0,B,Chvostek sign,Text,STEP 3,0.22,57.02,0.395388,54.353149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,612,A 45-year-old man is brought to the clinic by ...,Atrophy,Decreased visual acuity,Loss of proprioception,Presence of palmomental reflex,Ptosis,0,0,0,0,0,A,Atrophy,Text,STEP 1,0.22,95.17,0.367123,91.154839
663,315,A 64-year-old man with non-Hodgkin lymphoma co...,Bleomycin,Cyclophosphamide,Cytarabine,Doxorubicin,Fluorouracil,Methotrexate,Vincristine,0,0,0,G,Vincristine,Text,STEP 1,0.14,42.72,0.352678,44.828369
664,509,A 9-month-old boy is brought to the office by ...,Abdominal ultrasonography,Antiâ€“Saccharomyces cerevisiae antibody testing,Nitroblue tetrazolium testing,Rectal swab for group B streptococcus,Skeletal survey,Stool culture for Salmonella enteritidis,0,0,0,0,E,Skeletal survey,Text,STEP 2,0.94,122.73,0.498538,57.597900
665,550,A 32-year-old man comes to the office because ...,Collagen,Double-stranded DNA,Nucleolar protein,Phospholipid,Proteins in neutrophil cytoplasm,0,0,0,0,0,A,Collagen,Text,STEP 1,0.30,86.13,0.302066,60.293667


In [24]:
df.to_csv("Predicted_Difficulty_Response_time.csv")